## FTY Types for Hardware

The example uses FTY to define hardware structures:

```lisp
;; A signal identifier
(defprod sig
  ((module symbolp)
   (index integerp)))

;; A path of signals
(deflist sig-path :elt-type sig-p :true-listp t)

;; State: mapping from signal paths to boolean values
(defalist any-table
  :key-type sig-path-p
  :val-type booleanp
  :true-listp t)

;; A trace is a sequence of states
(deflist any-trace :elt-type any-table :true-listp t)
```

## Inverter Model

```lisp
(defprod inverter
  ((input sig-path-p)
   (output sig-path-p)))

;; An inverter step is valid if:
;; - If input != output in prev state, output can change
;; - If input == output in prev state and input changes, it's valid
;; - Stuttering (no change) is always allowed
(define inverter-valid-step ((i inverter-p)
                             (prev any-table-p)
                             (next any-table-p))
  :returns (ok booleanp)
  ...)

;; Count "markers" - how many input/output pairs match
(define inverter-count ((i inverter-p) (curr any-table-p))
  :returns (markers maybe-integer-p)
  ...)
```

## Ring Oscillator Structure

```lisp
(defprod ringosc3
  ((n1 sig-path-p)    ; node 1
   (n2 sig-path-p)    ; node 2  
   (n3 sig-path-p)    ; node 3
   (inv1 inverter-p)  ; inverter 1: n3 -> n1
   (inv2 inverter-p)  ; inverter 2: n1 -> n2
   (inv3 inverter-p)  ; inverter 3: n2 -> n3
   ))

;; Structural constraints: inverters connected correctly
(define ringosc3-constraints ((r ringosc3-p))
  :returns (ok booleanp)
  (and (equal (inverter->input (ringosc3->inv1 r)) (ringosc3->n3 r))
       (equal (inverter->output (ringosc3->inv1 r)) (ringosc3->n1 r))
       ...))
```

## Safety Property: "One Safe"

The key invariant is that exactly one inverter has matching input/output at any time:

```lisp
;; A state is "one safe" if exactly one inverter has in==out
(define ringosc3-one-safe-state ((r ringosc3-p)
                                 (curr any-table-p))
  :returns (ok booleanp)
  (b* ((count (ringosc3-count r curr))
       ((if (equal count (maybe-integer-fix nil))) nil))
    (equal (maybe-integer-some->val count) 1)))

;; The invariant holds for entire trace
(define ringosc3-one-safe-trace ((r ringosc3-p)
                                 (tr any-trace-p))
  :returns (ok booleanp)
  :measure (len tr)
  (b* (((unless (consp (any-trace-fix tr))) t)
       (first (car (any-trace-fix tr)))
       (rest (cdr (any-trace-fix tr)))
       ((unless (ringosc3-one-safe-state r first)) nil))
    (ringosc3-one-safe-trace r rest)))
```

## The Main Theorem

**Theorem:** If a ring oscillator starts in a "one safe" state and follows valid transitions, it stays "one safe".

```lisp
(defthm ringosc3-one-safe-lemma
  (implies (and (ringosc3-p r)
                (any-trace-p tr)
                (consp (any-trace-fix tr))
                (consp (any-trace-fix (cdr (any-trace-fix tr))))
                (ringosc3-constraints r)
                (inverter-valid-step (ringosc3->inv1 r) ...)
                (inverter-valid (ringosc3->inv1 r) ...)
                ;; ... similar for inv2 and inv3
                (ringosc3-one-safe-state r (car (any-trace-fix tr))))
           (ringosc3-one-safe-state r (car (any-trace-fix (cdr ...)))))
  :hints (("Goal"
           :smtlink
           (:fty (inverter ringosc3 any-trace any-table 
                  sig-path-list sig-path sig maybe-integer)
            :functions ((ringosc3-one-safe-trace ...) ...)))))
```

## SMTLink Hints for Hardware

Notice the rich `:smtlink` hints:

### FTY Types
```lisp
:fty (inverter ringosc3 any-trace any-table 
      sig-path-list sig-path sig maybe-integer)
```
All FTY types used in the theorem must be declared.

### Functions with Expansion Levels
```lisp
:functions ((ringosc3-one-safe-trace :formals ((r ringosc3-p)
                                               (tr any-trace-p))
                                     :returns ((ok booleanp))
                                     :level 1)
            (sigs-in-bool-table :formals ((sigs sig-path-listp)
                                          (st any-table-p))
                                :returns ((ok booleanp))
                                :level 2)
            (inverter-valid :formals (...) :level 0))
```

- **`:level 0`** - Uninterpreted (Z3 knows nothing about body)
- **`:level 1`** - Expand once
- **`:level 2`** - Expand twice

## Inductive Proof Structure

The final theorem uses induction combined with SMTLink:

```lisp
(defthm ringosc3-one-safe
  (implies (and (ringosc3-p r)
                (any-trace-p tr)
                (consp tr)
                (ringosc3-valid r tr)
                (ringosc3-one-safe-state r (car tr)))
           (ringosc3-one-safe-trace r tr))
  :hints (("Goal"
           :induct (ringosc3-one-safe-trace r tr)
           :in-theory (e/d (ringosc3-one-safe-trace
                           ringosc3-valid
                           inverter-valid)
                          (ringosc3-one-safe-lemma)))
          ("Subgoal *1/1.1"
           :use ((:instance ringosc3-one-safe-lemma ...)))))
```

### Strategy:

1. **Induction** on the trace structure
2. **SMTLink** proves the base case (one-step lemma)
3. **ACL2** handles the inductive step

## Why SMTLink for Hardware?

### Advantages:

1. **Case Analysis**: Z3 exhaustively checks all boolean combinations
2. **FTY Integration**: Structured types translate cleanly to Z3
3. **Automation**: Less manual lemma crafting than pure ACL2
4. **Counter-Examples**: Failed proofs show concrete traces

### Typical Workflow:

1. Define hardware with FTY types
2. Write transition relations
3. State invariant properties
4. Use SMTLink for case splits, ACL2 for induction

## Files Referenced

- `smtlink/examples/ringosc.lisp` - Ring oscillator example
- `smtlink/examples/inverter.lisp` - Inverter model
- `smtlink/examples/util.lisp` - FTY type definitions